# Tutorial for the Langchain functions chain
In this notebook you'll see the steps to run a chain that helps you to choose the right path depending on the query as provided by the user. To help you understand the concepts, we left out all the interaction with external data sources. We leave it up to your imagination to connect the different functions to real data.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Functions to call
Below are the three functions that should call the datasource that provides the answer to the question at hand.

In [2]:
def search_products(query: str) -> dict:
    """
    Searches for products using the provided keywords
    
    Args:
        query: The questions asked by the user in the form of a few keywords        
    """
    print(f"search_products: {query}")
    return {
        "products": [
            {"name": "Product 1", "price": "€ 12,00", "description": "The best product there is."},
            {"name": "Product 2", "price": "€ 23,00", "description": "The most expensive product there is."},
        ]
    }


def search_answer(question: str) -> dict:
    """
    Generates an answer to the provided question about our store like: account information, return policy, vision, values, sustainability.
    
    Args:
        question: The question the user wants an answer for 
    """
    print(f"search_answer: {question}")
    return {
        "answer": "The answer to all questions related to the galaxy is 42",
        "source_documents": [
            {
                "url": "https://en.wikipedia.org/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy"
            }
        ]
    }


def search_data(day_of_the_year: str) -> dict:
    """
    Obtain sales data for the provided day.

    Args:
        day_of_the_year: The day to obtain sales data for, format of the date string is yyyy-MM-dd
    """
    print(f"search_answer: {day_of_the_year}")
    return {
        "products_sold": 42,
        "customer_visits": 3223
    }

## Choose the function to call
Use a large language model decide which function to call.

In [4]:
from langchain.chains.openai_functions import create_openai_fn_chain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI


def init_fn_chain():
    llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=os.getenv('OPEN_AI_API_KEY'))
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a system to search for answers on a provided question."),
            ("human", "Make calls to the relevant function using the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

    return create_openai_fn_chain(
        functions=[search_products, search_answer, search_data],
        llm=llm,
        prompt=prompt,
        verbose=True
    )


## Initialise the chain
Create the chain and the code to call the python function using the result from the chain.

In [5]:
chain = init_fn_chain()

functions_map = {
    'search_products': search_products,
    'search_answer': search_answer,
    'search_data': search_data
}


def call_the_right_function(question: str):
    chain_response = chain.run(question)
    print(f"The response of the chain is {chain_response}")
    function_name = chain_response["name"]
    args = chain_response["arguments"]

    if function_name not in functions_map:
        chain_response["response"] = f"Could not decide which content search to perform for name '{function_name}'"
    else:
        chain_response["response"] = functions_map[function_name](**args)

    return chain_response


## Verify multiple questions
Uses the provided question and expected answer combinations to verify if the function is working as expected.

In [12]:
def verify_results(questions: list) -> list:
    results = []
    for question, correct_answer in questions:
        resulting_answer = call_the_right_function(question=question)
        results.append(f"{str(resulting_answer['name'] == correct_answer)}: {question}")
    return results


## Run the sample
Executes the chain using three different questions that all must be calling one of the three functions.

In [14]:
user_questions = [
    ("What books do you have about life hacking", "search_products"),
    ("how many books did we sell the 14th of July 2023", "search_data"),
    ("Do I need an account before I can purchase something?", "search_answer")
]

for answer in verify_results(user_questions):
    print(answer)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a system to search for answers on a provided question.
Human: Make calls to the relevant function using the following input: What books do you have about life hacking
Human: Tip: Make sure to answer in the correct format

> Finished chain.
The response of the chain is {'name': 'search_products', 'arguments': {'query': 'life hacking books'}}
search_products: life hacking books


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a system to search for answers on a provided question.
Human: Make calls to the relevant function using the following input: how many books did we sell the 14th of July 2023
Human: Tip: Make sure to answer in the correct format

> Finished chain.
The response of the chain is {'name': 'search_data', 'arguments': {'day_of_the_year': '2023-07-14'}}
search_answer: 2023-07-14


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a system to search 